In [1]:
%pip install -q mediapipe==0.10.7

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
import mediapipe as mp
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [3]:
filtered_hand = list(range(21))

filtered_pose = [11, 12, 13, 14, 15, 16]

filtered_face = [0, 4, 7, 8, 10, 13, 14, 17, 21, 33, 37, 39, 40, 46, 52, 53, 54, 55, 58,
                 61, 63, 65, 66, 67, 70, 78, 80, 81, 82, 84, 87, 88, 91, 93, 95, 103, 105,
                 107, 109, 127, 132, 133, 136, 144, 145, 146, 148, 149, 150, 152, 153, 154,
                 155, 157, 158, 159, 160, 161, 162, 163, 172, 173, 176, 178, 181, 185, 191,
                 234, 246, 249, 251, 263, 267, 269, 270, 276, 282, 283, 284, 285, 288, 291,
                 293, 295, 296, 297, 300, 308, 310, 311, 312, 314, 317, 318, 321, 323, 324,
                 332, 334, 336, 338, 356, 361, 362, 365, 373, 374, 375, 377, 378, 379, 380,
                 381, 382, 384, 385, 386, 387, 388, 389, 390, 397, 398, 400, 402, 405, 409,
                 415, 454, 466, 468, 473]

HAND_NUM = len(filtered_hand)
POSE_NUM = len(filtered_pose)
FACE_NUM = len(filtered_face)

In [4]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

def get_frame_landmarks(frame):
    
    all_landmarks = np.zeros((HAND_NUM * 2 + POSE_NUM + FACE_NUM, 3))
    
    def get_hands(frame):
        results_hands = hands.process(frame)
        if results_hands.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
                if results_hands.multi_handedness[i].classification[0].index == 0: 
                    all_landmarks[:HAND_NUM, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # right
                else:
                    all_landmarks[HAND_NUM:HAND_NUM * 2, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # left

    def get_pose(frame):
        results_pose = pose.process(frame)
        if results_pose.pose_landmarks:
            all_landmarks[HAND_NUM * 2:HAND_NUM * 2 + POSE_NUM, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])[filtered_pose]
        
    def get_face(frame):
        results_face = face_mesh.process(frame)
        if results_face.multi_face_landmarks:
            all_landmarks[HAND_NUM * 2 + POSE_NUM:, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark])[filtered_face]
        
    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.submit(get_hands, frame)
        executor.submit(get_pose, frame)
        executor.submit(get_face, frame)

    return all_landmarks

In [5]:
def get_video_landmarks(video_path, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(video_path)
    
    # if the starting is 0
    if start_frame <= 1:
        start_frame = 1
        
    # if the video is precropped
    elif start_frame > int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
        start_frame = 1
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    # if the final frame was not given (-1)    
    if end_frame < 0: 
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    num_landmarks = HAND_NUM * 2 + POSE_NUM + FACE_NUM
    all_frame_landmarks = np.zeros((end_frame - start_frame + 1, num_landmarks, 3))
    frame_index = 1
    
    while cap.isOpened() and frame_index <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index >= start_frame:
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_landmarks = get_frame_landmarks(frame)
            all_frame_landmarks[frame_index - start_frame] = frame_landmarks

        frame_index += 1

    cap.release()
    hands.reset()
    pose.reset()
    face_mesh.reset()
    return all_frame_landmarks

In [6]:
def draw_landmarks(input_path, output_path, video_landmarks, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    if start_frame <= 1:
        start_frame = 1
    elif start_frame > int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
        start_frame = 1
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if end_frame < 0:
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    frame_index = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index >= start_frame and frame_index <= end_frame:
            frame_landmarks = video_landmarks[frame_index - start_frame]
            landmarks = [(int(x * width), int(y * height)) for x, y, _ in frame_landmarks]
            for x, y in landmarks:
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
            out.write(frame)
        else:
            # out.write(frame) # Enable if you want the full video
            pass
        frame_index += 1

    cap.release()
    out.release()

In [7]:
npy_dir = 'landmarks'
os.makedirs(npy_dir, exist_ok=True)

In [8]:
with open('WLASL_data.json', 'r') as json_file:
    data = json.load(json_file)

In [9]:
co = {}
try:
    for i in tqdm(range(len(data)), ncols=100):
        gloss = data[i]['gloss']
        if gloss not in co:
            co[gloss] = 1
        else:
            co[gloss] += 1
        
        npy_path = os.path.join(npy_dir, f"{gloss}{co[gloss]}.npy")
        if os.path.exists(npy_path):
            continue
        
        video_path = data[i]['video_path']
        start = data[i]['frame_start']
        end = data[i]['frame_end']
        
        try:
            video_landmarks = get_video_landmarks(video_path, start, end)
            np.save(npy_path, video_landmarks)
            
        except Exception as e:
            print(f"\nError encoding {video_path}\n{e}")
            continue
        
        clear_output(wait=True)

except KeyboardInterrupt:
    print("\nLoading process interrupted by user.")


100%|██████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 28642.77it/s]


In [10]:
landmarks_dict = {}

for filename in os.listdir(npy_dir):
    if filename.endswith('.npy'):
        key = filename.split('.')[0]
        landmarks = np.load(os.path.join(npy_dir, filename), allow_pickle=True)
        landmarks_dict[key] = landmarks

In [11]:
# Load the word list
with open('wordlist.txt', 'r') as f:
    word_index = {word:index for index, word in enumerate([line.strip() for line in f])}
    reverse_word_index = {index: word for word, index in word_index.items()}

In [12]:
import re

X, y = [], []
for i in landmarks_dict:
    for j in landmarks_dict[i]:
        X.append(j)
        y.append(np.eye(len(word_index), dtype=int, k=word_index[re.sub(r'\d+$', '', i)])[0])
X, y = np.array(X), np.array(y)
        

In [13]:
# Split the data to Train, and Test (80%, 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(512, input_shape=X.shape[1:], activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(len(word_index), activation='softmax'))

initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=10000, decay_rate=0.9)
early_stopping = EarlyStopping(
    monitor='loss',  # Metric to monitor for early stopping
    mode='min',  # Set mode to 'min' for minimizing the metric
    patience=15,  # Number of epochs with no improvement before stopping
    restore_best_weights=True,  # Restore the best model weights
    verbose=1
)

model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 180, 512)          2048      
                                                                 
 dropout (Dropout)           (None, 180, 512)          0         
                                                                 
 dense_1 (Dense)             (None, 180, 256)          131328    
                                                                 
 dropout_1 (Dropout)         (None, 180, 256)          0         
                                                                 
 dense_2 (Dense)             (None, 180, 128)          32896     
                                                                 
 dropout_2 (Dropout)         (None, 180, 128)          0         
                                                                 
 flatten (Flatten)           (None, 23040)             0

In [16]:
# Train model
model_training_history = model.fit(X_train, y_train, batch_size=128, epochs=30 , callbacks=[early_stopping])

Epoch 1/30
101/101 [==============================] - 28s 275ms/step - loss: 2.2096 - accuracy: 0.2822
Epoch 2/30
101/101 [==============================] - 27s 270ms/step - loss: 1.6581 - accuracy: 0.4667
Epoch 3/30
101/101 [==============================] - 27s 270ms/step - loss: 1.4265 - accuracy: 0.5473
Epoch 4/30
101/101 [==============================] - 28s 272ms/step - loss: 1.2892 - accuracy: 0.5839
Epoch 5/30
101/101 [==============================] - 27s 271ms/step - loss: 1.2013 - accuracy: 0.6148
Epoch 6/30
101/101 [==============================] - 28s 279ms/step - loss: 1.1268 - accuracy: 0.6352
Epoch 7/30
101/101 [==============================] - 27s 271ms/step - loss: 1.0719 - accuracy: 0.6495
Epoch 8/30
101/101 [==============================] - 27s 272ms/step - loss: 1.0409 - accuracy: 0.6586
Epoch 9/30
101/101 [==============================] - 27s 272ms/step - loss: 0.9979 - accuracy: 0.6673
Epoch 10/30
101/101 [==============================] - 27s 270ms/step - l

In [17]:
# Evaluate the model on train data
print('Train Loss: {:.6f}, Accuracy: {:.6f}'.format(*model.evaluate(X_train, y_train, verbose=0)))

Train Loss: 0.377358, Accuracy: 0.876121


In [18]:
# Evaluate the model on test data
print('Test Loss: {:.6f}, Accuracy: {:.6f}'.format(*model.evaluate(X_test, y_test, verbose=0)))
model_evaluation_history = model.evaluate(X_test, y_test, verbose=0)

Test Loss: 0.434277, Accuracy: 0.859282


Real-time section

In [23]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Get the height and width of the frame
    height, width, _ = frame.shape

    # Get landmarks for the frame
    frame_landmarks = get_frame_landmarks(frame)

    # Draw landmarks on the frame
    for landmark in frame_landmarks:
        x = int(landmark[0] * width)
        y = int(landmark[1] * height)
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    frame_pred = model.predict(np.array([frame_landmarks]), verbose=0)
    frame_word_index = np.argmax(frame_pred)
    probability = round(np.max(frame_pred)*100,2)
    
    # Add text to the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, reverse_word_index[frame_word_index] + ': ' + str(probability) + '%', (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('Frame with Landmarks', frame)

    # Check for the 'q' key to quit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

